In [0]:
customer_df = spark.read\
    .option("header", True)\
    .option("inferSchema", True)\
    .csv("abfss://bronze@datalakeecomm.dfs.core.windows.net/olist_customers_dataset.csv")

In [0]:
display(customer_df)

In [0]:
geolocation_df = spark.read\
    .option("header", True)\
    .option("inferSchema", True)\
    .csv("abfss://bronze@datalakeecomm.dfs.core.windows.net/olist_geolocation_dataset.csv")

In [0]:
order_items_df = spark.read\
    .option("header", True)\
    .option("inferSchema", True)\
    .csv("abfss://bronze@datalakeecomm.dfs.core.windows.net/olist_order_items_dataset.csv")



In [0]:
order_payments_df = spark.read\
    .option("header", True)\
    .option("inferSchema", True)\
    .csv("abfss://bronze@datalakeecomm.dfs.core.windows.net/olist_order_payments_dataset.csv")

In [0]:
order_reviews_df = spark.read\
    .option("header", True)\
    .option("inferSchema", True)\
    .csv("abfss://bronze@datalakeecomm.dfs.core.windows.net/olist_order_reviews_dataset.csv")

In [0]:
orders_dataset_df = spark.read\
    .option("header", True)\
    .option("inferSchema", True)\
    .csv("abfss://bronze@datalakeecomm.dfs.core.windows.net/olist_orders_dataset.csv")

In [0]:
products_dataset_df = spark.read\
    .option("header", True)\
    .option("inferSchema", True)\
    .csv("abfss://bronze@datalakeecomm.dfs.core.windows.net/olist_products_dataset.csv")

In [0]:
sellers_dataset_df = spark.read\
    .option("header", True)\
    .option("inferSchema", True)\
    .csv("abfss://bronze@datalakeecomm.dfs.core.windows.net/olist_sellers_dataset.csv")


In [0]:
from pymongo import MongoClient

In [0]:
# importing module
from pymongo import MongoClient
import pandas as pd

hostname = "vjgs2n.h.filess.io"
database = "ecomdatanosql_downrodor"
port = "27018"
username = "ecomdatanosql_downrodor"
password = "5454ea959608a1302cee10ee88a0a5e15e31611e"

uri = "mongodb://" + username + ":" + password + "@" + hostname + ":" + port + "/" + database

# Connect with the portnumber and host
client = MongoClient(uri)

# Access database
mydatabase = client[database]
mydatabase


In [0]:
collection = mydatabase['product_categories']

mongo_data = pd.DataFrame(list(collection.find()))


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
def clean_DataFrame(df, name):
    print('Cleaning ' + name)
    return df.dropDuplicates().na.drop('all')

In [0]:
    clean_DataFrame(customer_df, 'customer_Data')
    clean_DataFrame(geolocation_df, 'geolocation_Data')
    clean_DataFrame(order_items_df, 'order_items_Data')
    clean_DataFrame(order_payments_df, 'order_payments_Data')
    clean_DataFrame(order_reviews_df, 'order_reviews_Data')
    clean_DataFrame(orders_dataset_df, 'orders_dataset_Data')
    clean_DataFrame(products_dataset_df, 'products_dataset_Data')
    clean_DataFrame(sellers_dataset_df, 'sellers_dataset_Data')

display(customer_df)
display(geolocation_df)
display(order_items_df)
display(order_payments_df)
display(order_reviews_df)
display(orders_dataset_df)
display(products_dataset_df)
display(sellers_dataset_df)


In [0]:
import pandas as pd

# Convert ObjectId to string
mongo_data['_id'] = mongo_data['_id'].astype(str)

# Create DataFrame
mongo_df = spark.createDataFrame(mongo_data)

clean_DataFrame(mongo_df, 'mongo_data')
display(mongo_df)

In [0]:
display(customer_df)
display(geolocation_df)
display(order_items_df)
display(order_payments_df)
display(order_reviews_df)
display(orders_dataset_df)
display(products_dataset_df)
display(sellers_dataset_df)

In [0]:
from pyspark.sql.functions import *

## Convert Date Columns
orders_dataset_df = orders_dataset_df.withColumn(
    "order_purchase_timestamp", to_date(col("order_purchase_timestamp"))
).withColumn(
    "order_approved_at", to_date(col("order_approved_at"))
).withColumn(
    "order_delivered_carrier_date", to_date(col("order_delivered_carrier_date"))
).withColumn(
    "order_delivered_customer_date", to_date(col("order_delivered_customer_date"))
).withColumn(
    "order_estimated_delivery_date", to_date(col("order_estimated_delivery_date"))
)

display(orders_dataset_df)

In [0]:
# Calculate Time and Delivery Delay

orders_dataset_df = orders_dataset_df.withColumn('actual_delivery_date', datediff(col('order_delivered_customer_date'), col('order_purchase_timestamp')))

orders_dataset_df = orders_dataset_df.withColumn('estimated_delivery_time', date_diff('order_estimated_delivery_date', 'order_purchase_timestamp'))

orders_dataset_df = orders_dataset_df.withColumn('delay', col('actual_delivery_date') - col('estimated_delivery_time'))

orders_dataset_df = orders_dataset_df.withColumn('is_delayed', when(col('delay') < 0, 'False').otherwise('True'))

display(orders_dataset_df)


In [0]:
display(customer_df)
display(geolocation_df)
display(order_items_df)
display(order_payments_df)
display(order_reviews_df)
display(orders_dataset_df)
display(products_dataset_df)
display(sellers_dataset_df) 

In [0]:
order_payment_df = orders_dataset_df.join(order_payments_df, orders_dataset_df.order_id == order_payments_df.order_id).drop(order_payments_df.order_id)

order_payment_df = order_payment_df.select([col for col in order_payment_df.columns if order_payment_df.columns.count(col) == 1])

order_pay_cust_df = order_payment_df.join(customer_df, order_payment_df.customer_id == customer_df.customer_id).drop(order_payment_df.customer_id)

order_pay_cust_df = order_pay_cust_df.select([col for col in order_pay_cust_df.columns if order_pay_cust_df.columns.count(col) == 1])




In [0]:
orders_items_df = order_pay_cust_df.join(order_items_df, order_pay_cust_df.order_id == order_items_df.order_id).drop(order_items_df.order_id)

orders_items_df = orders_items_df.select([col for col in orders_items_df.columns if orders_items_df.columns.count(col) == 1])


product_item_df = orders_items_df.join(products_dataset_df,orders_items_df.product_id ==  products_dataset_df.product_id).drop(products_dataset_df.product_id )

product_item_df = product_item_df.select([col for col in product_item_df.columns if product_item_df.columns.count(col) == 1])

In [0]:
order_seller = product_item_df.join(sellers_dataset_df,product_item_df.seller_id == sellers_dataset_df.seller_id).drop(sellers_dataset_df.seller_id)

order_seller = order_seller.select([col for col in order_seller.columns if order_seller.columns.count(col) == 1])

In [0]:
seller_geolocation_df = order_seller.join(geolocation_df, order_seller.seller_zip_code_prefix == geolocation_df.geolocation_zip_code_prefix).drop(geolocation_df.geolocation_zip_code_prefix)

seller_geolocation_df = seller_geolocation_df.select([col for col in seller_geolocation_df.columns if seller_geolocation_df.columns.count(col) == 1])

In [0]:
final_df = seller_geolocation_df.join(order_reviews_df, order_reviews_df.order_id == seller_geolocation_df.order_id).drop(order_reviews_df.order_id)

final_df = final_df.select([col for col in final_df.columns if final_df.columns.count(col) == 1])


In [0]:
display(final_df)

In [0]:
display(final_df)

Databricks data profile. Run in Databricks to view.

In [0]:
display(final_df)

Databricks visualization. Run in Databricks to view.

In [0]:
final_df.repartition(1).write.mode("overwrite").parquet('abfss://silver@datalakeecomm.dfs.core.windows.net/')

In [0]:
final_df.write.mode("overwrite").parquet('abfss://silver@datalakeecomm.dfs.core.windows.net/Delta_Table')